**Prerequisites to running this notebook:**
1. Install the following in your dev environment:<br>
    a. google-cloud-bigquery: pip.exe install google-cloud-bigquery<br>
    b. db-types: pip install db-dtypes<br>
2. Install gcloud CLI <br>
    a. Install directions (with download link): https://cloud.google.com/sdk/docs/install<br>
    > i. pay attention to where it installs to add this to your PATH environmental variables<br>
    > ii. It says to leave all the shortcut, open terminal options checked. I received errors when it ran "gcloud info --run-diagnostics" and I ignored them until I updated the PATH envrionmental variables<br>
    
    b. Add this to your PATH environmental variables (for me this was C:\Users\vt_be\AppData\Local\Google\Cloud SDK\google-cloud-sdk)<br>
    c. reboot!<br>
    d. open git bash, switch to dev environment<br>
    > i. "gcloud info --run-diagnostics" now ran without issue<br>
    ii. add authentication (this opens browser to connect your google account):  gcloud auth application-default login<br>
    
    e. I also needed to set up a Big Query Project: mostly followed https://cloud.google.com/bigquery/docs/sandbox<br>
    > i. I didn't see the stuff mentioned in #3 but it otherwise worked as written<br>
    > ii. Note that when you create the project, an id is generated that is project name - #### (for me BootCamp-Weather:  bootcamp-weather-400118<br>
    
    f. Add the project to default - back to gitbash: gcloud auth application-default set-quota-project <project-id><br>
3. Create a config.py file in the folder that contains this notebook. 
4. Add the following variable to config.py: google_project = "your-project-id"
    

**Credit:**
* Big Query calls adapted from https://www.kaggle.com/code/crained/noaa-dataset-with-google-bigquery
* SQL calls adapted from GitHub BigQuery documentation: https://github.com/googleapis/python-bigquery

**Data Info:**
https://data.noaa.gov/dataset/dataset/global-surface-summary-of-the-day-gsod

In [1]:
# Since this is unique to user, I added config.py to the gitignore. 
# You must create your own config.py file with project name as stated in the Prerequisites
from config import google_project
# bigquery and pandas work well together for dataframes!
import pandas as pd
import os
# Follow the prerequisite instructions to get bigquery going
from google.cloud import bigquery
# Create a "Client" object reference a google project for which your system has been authenticated
client = bigquery.Client(google_project)

**Optional print the table schemas to better understand the data** <br>
Commented out here due to length of output for complete notebook execution

In [2]:
# # Stations table
# # Construct a reference to the "full" table
# table_ref2 = dataset_ref.table("stations")
# # API request - fetch the table
# table2 = client.get_table(table_ref2)
# # Print information on all the columns
# table2.schema

In [3]:
# # Measurements table for 2022
# # Construct a reference to the "full" table
# table_ref = dataset_ref.table("gsod2022")
# # API request - fetch the table
# table = client.get_table(table_ref)
# # Print information on all the columns
# table.schema

## Query for the Stations static info ##
usaf for connecting tables, <br>
spatial data required for heatmap, <br>
name and state for usability <br>

In [4]:
# Station IDs (usaf) remained consistent to the physical station over time, but names changed resulting in multiple entries with same usaf. 
# This query will sort by begin date descending so we can use the pandas remove duplicates function to keep the latest name only.

# Define the SQl suery
QUERY = (
    'SELECT usaf, name, state, lat, lon, elev '
    'FROM `bigquery-public-data.noaa_gsod.stations` '
    'WHERE country = "US" AND state <> "None" '
    'ORDER BY begin DESC'
    )
# API request
stations_result = client.query(QUERY)  
# Waits for query to finish
stations_data = stations_result.result()  

# Put the query into a dataframe
stations_data_df = stations_data.to_dataframe()

# Remove duplicate stations by usaf
stations_data_df = stations_data_df.drop_duplicates("usaf")

# # and export if desired (not required for final implementation)
# stations_data_df.to_json("./data/Stations.json", orient="records")
# stations_data_df.to_csv("./data/Stations.csv")

print(f' There are {len(stations_data_df)} rows')
stations_data_df.head()

 There are 3790 rows


,usaf,name,state,lat,lon,elev
0,720511,BRITTON MUNI,SD,45.815,-97.743,+0401.7
1,723062,PINEY ISLAND,NC,35.020,-76.460,+0005.2
2,A00030,CONNELLSVILLE AIRPORT,PA,39.959,-79.657,+0386.2
3,720844,SPANISH PEAKS,CO,37.697,-104.785,+1844.0
4,724916,MARINA MUNI,CA,36.682,-121.762,+0040.8


## Query for the temperature data ##
### Query for temperature stats (similar filtering) by station ###
SchemaField('min', 'FLOAT', 'NULLABLE', None, 'Minimum temperature reported during the day in Fahrenheit to tenths--time of min temp report varies by country and region, so this will sometimes not be the min for the calendar day. Missing = 9999.9', (), None),<br>
SchemaField('temp', 'FLOAT', 'NULLABLE', None, 'Mean temperature for the day in degrees Fahrenheit to tenths. Missing = 9999.9', (), None),<br>
SchemaField('max', 'FLOAT', 'NULLABLE', None, 'Maximum temperature reported during the day in Fahrenheit to tenths--time of max temp report varies by country and region, so this will sometimes not be the max for the calendar day. Missing = 9999.9', (), None),

In [5]:
# Define the query to get absoulte minimum temperature, average mean temperature, absolute maximum temperature 
# for the year by station (usaf)
aggregate_query = (
    'SELECT s.usaf, '
    'MIN(g.min) AS min_temp, '
    'AVG(g.temp) AS mean_temp, '
    'MAX(g.max) AS max_temp, '
    'FROM `bigquery-public-data.noaa_gsod.gsod2022` AS g '
    'INNER JOIN `bigquery-public-data.noaa_gsod.stations` AS s ON g.stn = s.usaf '
    'WHERE s.country = "US" AND s.state <> "None" '
    # The line below removes the 'not a reading' so we can run stats on those columns
    'AND g.min <> 9999.9 AND g.max <> 9999.9 '
    'GROUP BY s.usaf '
    )

# API request
station_temp_result = client.query(aggregate_query)  
# Waits for query to finish
station_temp_data = station_temp_result.result()  

# Put the query into a dataframe
station_temp_df = station_temp_result.to_dataframe()

# # and export if desired (not required for final implementation)
# state_temp_station.to_json("./data/Station_temp_sample.json", orient="records")
# state_temp_station.to_csv("./data/Station_temp_sample.csv")

print(f' There are {len(station_temp_df)} rows')
station_temp_df.head()

 There are 2522 rows


,usaf,min_temp,mean_temp,max_temp
0,702490,-27.4,32.636119,77.0
1,702606,-7.6,24.352830,55.4
2,703333,21.9,45.812844,69.1
3,703406,-22.0,42.212132,82.4
4,703656,-32.8,35.676400,80.1


### Query for total precipitation in a year ###
SchemaField('prcp', 'FLOAT', 'NULLABLE', None, "Total precipitation (rain and/or melted snow) reported during the day in inches and hundredths; will usually not end with the midnight observation--i.e., may include latter part of previous day.  .00 indicates no measurable precipitation (includes a trace). Missing = 99.99 Note: Many stations do not report '0' on days with no precipitation--therefore, '99.99' will often appear on these days. Understand this to see if need to include a filter around teh flag
Also, for example, a station may only report a 6-hour amount for the period during which rain fell. See Flag field for source of data", (), None),


 SchemaField('flag_prcp', 'STRING', 'NULLABLE', None, "A = 1 report of 6-hour precipitation amount B = Summation of 2 reports of 6-hour precipitation amount C = Summation of 3 reports of 6-hour precipitation amount D = Summation of 4 reports of 6-hour precipitation amount E = 1 report of 12-hour precipitation amount F = Summation of 2 reports of 12-hour precipitation amount G = 1 report of 24-hour precipitation amount H = Station reported '0' as the amount for the day (eg, from 6-hour reports), but also reported at least one occurrence of precipitation in hourly observations--this could indicate a trace occurred, but should be considered as incomplete data for the day. I = Station did not report any precip data for the day and did not report any occurrences of precipitation in its hourly observations--it's still possible that precip occurred but was not reported", (), None),

**Due to the different ways stations are allowed to take measurements, there are duplicate readings in a single day for some stations** <br>
In particular, there is one station (999999) that behaves differently than the rest but within the schema that requires further clean-up.<br>
To work through this, a more detailed query is need that returns millions of rows taking quite a bit of time. This query was executed and then duplicates removed and stored in a csv and json files so the query would not have to be run again. The query is commented out and the re-load of the csv file is kept for default running of this notebook.

In [6]:
# # Perform a query for the sum of precipitation for each station over the year
# aggregate_query = (
#     'SELECT s.usaf, g.mo, g.da, g.prcp, g.flag_prcp '
#     # 'SUM(g.prcp) AS total_precipitation '
#     'FROM `bigquery-public-data.noaa_gsod.gsod2022` AS g '
#     'INNER JOIN `bigquery-public-data.noaa_gsod.stations` AS s ON g.stn = s.usaf '
#     'WHERE s.country = "US" AND s.state <> "None" '
#     # The line below removes the 'not a reading' so we can run stats on those columns
#     'AND g.prcp <> 99.99 '
#     # 'GROUP BY s.usaf'
# )
# station_prcp_result = client.query(aggregate_query)  # API request
# station_prcp_data = station_prcp_result.result()  # Waits for query to finish

# # Put the query into a dataframe
# station_prcp_long_df = station_prcp_result.to_dataframe()

# # An export at this level produces files at the gigabyte level so it is not recommended
# # station_prcp_long_df.to_csv("stations_rain_with_dup.csv")
# # station_prcp_long_df.to_json("stations_rain_with_dup.json", orient="records")

# # Drop duplicates across usaf	mo	da	prcp (duplicates are allowed by multiple flags)
# # Remove duplicate station day reports
# station_prcp_long_nodup = station_prcp_long_df.drop_duplicates(["usaf", "mo", "da", "prcp"])

# station_prcp_long_nodup.to_csv("../data/stations_prcp_detail.csv")
# station_prcp_long_nodup.to_json("../data/stations_prcp_detail.json", orient="records")

# print(f' There are {len(station_prcp_long_nodup)} rows')
# station_prcp_long_nodup.head()

In [7]:
# Read in the file exported in the commented code to the same dataframe.
# If you chose to run the above cell, this cell may be commented out.
station_prcp_long_nodup = pd.read_csv('../data/stations_prcp_detail.csv')

In [8]:
# Removing overlapping readings (station 99999 for the 2022 pull but could be more stations in other years)

# Sort the stations by station -> month -> day -> prcp in descending order. 
# This puts the highest measurement for the day at the top so when we use panda's remove duplicates we keep it
# This method has an assumption that we highest measurment in the day is the last, but that is not necessarily true because the flag indicates 24 hours not "day"
# However, the schema does indicate a way to get the time to properly filter for non-overlapping readings
station_prcp_long_nodup = station_prcp_long_nodup.sort_values(by=['usaf', 'mo', 'da', 'prcp'], ascending=False)

# this view let me confirm what I was getting for the problematic station (999999)
# station_prcp_long_nodup2.iloc[9000:9050]

station_prcp_long_nodup = station_prcp_long_nodup.drop_duplicates(["usaf", "mo", "da"])

#### Get the total precipitation by station for the year ###

In [9]:
# Group by station and use sum to get total rain for the year and convert series to dataframe
station_prcp_total = station_prcp_long_nodup.groupby(["usaf"])["prcp"].sum().reset_index()

# Change column name
station_prcp_total.rename(columns = {'prcp':'total_precipitation'}, inplace = True)

# Sort by rain total
station_prcp_total = station_prcp_total.sort_values(by=['total_precipitation'], ascending=False)

print(f' There are {len(station_prcp_total)} rows')
station_prcp_total.head()

 There are 2522 rows


,usaf,total_precipitation
2486,999999,809.88
138,703950,155.33
116,703610,142.47
124,703710,102.80
2045,727970,97.15


#### Get the number of days with precipitation > 6in by station for the year ###

In [10]:
# Filter stations to only include precipitation over 6in and keep only the station and precipitation columns
station_prcp_filter = station_prcp_long_nodup[station_prcp_long_nodup["prcp"] > 6]
station_prcp_filter = station_prcp_filter[["usaf", "prcp"]]

# Group by station and count how many there are
station_prcp_count = station_prcp_filter.groupby(["usaf"]).count().reset_index()

station_prcp_count.head()

# Change column name
station_prcp_count.rename(columns = {'prcp':'high_prcp_days'}, inplace = True)
# Sort descending to gut check top of list
station_prcp_count = station_prcp_count.sort_values(by=['high_prcp_days'], ascending=False)

print(f' There are {len(station_prcp_count)} rows')
station_prcp_count.head()

 There are 27 rows


,usaf,high_prcp_days
26,999999,8
1,720357,2
0,720282,2
2,720384,2
3,720401,1


### Query for count of days with 6+ inches of snow in a year ###
SchemaField('sndp', 'FLOAT', 'NULLABLE', None, "Snow depth in inches to tenths--last report for the day if reported more thanonce. Missing = 999.9 Note: Most stations do not report '0' ondays with no snow on the ground--therefore, '999.9' will often appear on these days", (), None)

The snow depth is how much snow is on the ground at the time the station takes a measurement, not the amount of new snow.
We chose to approximate snowfall by taking the difference of snow depth day day n - day (n-1) using the maximum snow depth reading per day

In [11]:
# Perform a query for the sum of snow depth and count of days with 6 or more inches for each station over the year
# Some stations have multiple readings within a day but we do not have a timestamp to filter for exact day-to-day
# Instead we pull the max reading in a day for the approximation
aggregate_query = '''
WITH SnowDepthChanges AS (
  SELECT DISTINCT
    s.usaf,  -- Replace "stn" with "usaf"
    g.date,
    MAX(g.sndp) AS max_snow_depth,
    CASE WHEN MAX(g.sndp) >= 6.0 THEN 1 ELSE 0 END AS saw_6_or_more_inches
  FROM `bigquery-public-data.noaa_gsod.gsod2022` AS g
  JOIN `bigquery-public-data.noaa_gsod.stations` AS s ON g.stn = s.usaf
  WHERE s.country = "US" AND s.state <> "None" 
  GROUP BY s.usaf, g.date  -- Replace "stn" with "usaf"
)

SELECT
  usaf,  -- Replace "stn" with "usaf"
  SUM(CASE WHEN snow_depth_change >= 6.0 THEN 1 ELSE 0 END) AS high_sndp_change_days
FROM (
  SELECT
    usaf,  -- Replace "stn" with "usaf"
    date,
    max_snow_depth,
    CASE
      WHEN LAG(date, 1) OVER (PARTITION BY usaf ORDER BY date) IS NOT NULL THEN
        max_snow_depth - LAG(max_snow_depth, 1, 0) OVER (PARTITION BY usaf ORDER BY date)
      ELSE 0  -- Set the snow_depth_change to 0 for the first day of the year
    END AS snow_depth_change
  FROM SnowDepthChanges
) AS Subquery
GROUP BY usaf  -- Replace "stn" with "usaf"
ORDER BY high_sndp_change_days
'''

station_snow_result = client.query(aggregate_query)  # API request
station_snow_data = station_snow_result.result()  # Waits for query to finish

# Put the query into a dataframe
six_inches_snow_df = station_snow_result.to_dataframe()

print(f' There are {len(six_inches_snow_df)} rows')
six_inches_snow_df.head()

 There are 2523 rows


,usaf,high_sndp_change_days
0,720854,0
1,720993,0
2,722008,0
3,722061,0
4,722160,0


### Query for count of days with tornadoes in a year ###
SchemaField('tornado_funnel_cloud', 'STRING', 'NULLABLE', None, 'Indicators (1 = yes, 0 = no/not reported) for the occurrence during the day', (), None)

In [12]:
# Define the query to get the number of days with tornadoes for each station (usaf)
aggregate_query = (
    'SELECT s.usaf, '
    # Some stations have more than one reading in a day but that could still refer to a single event 
    # so we refer to the number of days with tornadoes rather than the number of tornadoes and do a distinct filter
    'COUNT(DISTINCT CASE WHEN g.tornado_funnel_cloud = "1" THEN DATE(g.date) ELSE NULL END) AS days_with_tornado '
    'FROM `bigquery-public-data.noaa_gsod.gsod2022` AS g '
    'INNER JOIN `bigquery-public-data.noaa_gsod.stations` AS s ON g.stn = s.usaf '
    'WHERE s.country = "US" AND s.state <> "None" '
    'GROUP BY s.usaf '
    'ORDER BY days_with_tornado DESC'
)

# API request
station_tornado_result = client.query(aggregate_query)
station_tornado_data = station_tornado_result.result()

# Put the query into a dataframe
station_tornado_df = station_tornado_result.to_dataframe()

print(f'There are {len(station_tornado_df)} rows')
station_tornado_df.head()

There are 2523 rows


,usaf,days_with_tornado
0,722015,8
1,722010,4
2,722039,4
3,722103,2
4,722020,2


### Query for count of days with hail in a year ###
SchemaField('hail', 'STRING', 'NULLABLE', None, 'Indicators (1 = yes, 0 = no/not reported) for the occurrence during the day', (), None)

In [13]:
# Define the query to get the number of days with hail for each station (usaf)
aggregate_query = (
    'SELECT s.usaf, '
    # Some stations have more than one reading in a day but that could still refer to a single event 
    # so we refer to the number of days with hail rather than the number of hail events and do a distinct filter
    'COUNT(DISTINCT CASE WHEN g.hail = "1" THEN DATE(g.date) ELSE NULL END) AS days_with_hail '
    'FROM `bigquery-public-data.noaa_gsod.gsod2022` AS g '
    'INNER JOIN `bigquery-public-data.noaa_gsod.stations` AS s ON g.stn = s.usaf '
    'WHERE s.country = "US" AND s.state <> "None" '
    'GROUP BY s.usaf '
    'ORDER BY days_with_hail DESC'
)

# API request
station_hail_result = client.query(aggregate_query)
station_hail_data = station_hail_result.result()

# Put the query into a dataframe
station_hail_df = station_hail_result.to_dataframe()

print(f'There are {len(station_hail_df)} rows')
station_hail_df.head()

There are 2523 rows


,usaf,days_with_hail
0,724057,27
1,704140,12
2,726510,5
3,726770,5
4,742060,5


## Merge all the data frames together ##

In [14]:
# Use an inner here to create a basis for all possible without adding NaN rows
station_temp_merge = pd.merge(stations_data_df, station_temp_df, how ="inner", on = "usaf")

# Use outer here to include everything as the some remaining dataframes do not
# include stations that did not register readings but have valid temperature readings
st_temp_prcp_merge = pd.merge(station_temp_merge, station_prcp_total, how ="outer", on = "usaf")

st_temp_prcp_merge2 = pd.merge(st_temp_prcp_merge, station_prcp_count, how ="outer", on = "usaf")

st_temp_prcp_snow = pd.merge(st_temp_prcp_merge2, six_inches_snow_df, how ="outer", on = "usaf")

st_temp_prcp_snow_torn = pd.merge(st_temp_prcp_snow, station_tornado_df, how ="outer", on = "usaf")

st_temp_prcp_snow_torn_hail = pd.merge(st_temp_prcp_snow_torn, station_hail_df, how ="outer", on = "usaf")

station_all = st_temp_prcp_snow_torn_hail

# Looking at the csv file, some queries brought in an unknown station with data values of 0
# Removing unkown station
station_all = station_all.dropna(subset=["name"])


# Convert the remaining NaN to zero for reporting and future math
station_all = station_all.fillna(0)

# Sort by states and name for easy dropdown menu population
station_all = station_all.sort_values(by=["state", "name"], ascending = [True, True])

# # and export if desired at this point
# station_all.to_json("./data/stations_all.json", orient="records")
# station_all.to_csv("./data/stations_all.csv")
# station_all.to_json("./data/stations_all.js", orient="records")

print(f' There are {len(station_all)} rows')
station_all.head()

 There are 2522 rows


,usaf,name,state,lat,lon,elev,min_temp,mean_temp,max_temp,total_precipitation,high_prcp_days,high_sndp_change_days,days_with_tornado,days_with_hail
2000,704540,ADAK (NAS),AK,51.883,-176.650,+0005.0,12.9,41.957418,69.1,46.22,0.0,0,0,0
682,997380,ADAK ISLAND,AK,51.870,-176.630,+0007.0,18.7,42.015254,64.9,0.00,0.0,0,0,0
738,703926,AKHIOK,AK,56.933,-154.183,+0013.0,8.1,37.401887,71.1,9.28,0.0,0,0,0
547,702686,AKIAK,AK,60.903,-161.231,+0009.1,-20.0,34.427976,75.9,1.45,0.0,0,0,0
13,999999,ALEKNAGIK 1 NNE,AK,59.284,-158.615,+0024.4,-57.1,52.816638,122.4,809.88,8.0,0,0,0


### Add Severity rating ###
We chose to weight tornadoes heavily due to potential damage

In [15]:
station_all["severity_rating"] = station_all["days_with_tornado"]*4 + station_all["days_with_hail"] + station_all["high_prcp_days"] + station_all["high_sndp_change_days"]

# and export
station_all.to_json("../data/stations_all.json", orient="records")
station_all.to_csv("../data/stations_all.csv")
station_all.to_json("../data/stations_all.js", orient="records")

# Sort by severity for viewing purposes only to verify math
# station_all = station_all.sort_values(by=['severity_rating'], ascending=False)

station_all.head()

,usaf,name,state,lat,lon,elev,min_temp,mean_temp,max_temp,total_precipitation,high_prcp_days,high_sndp_change_days,days_with_tornado,days_with_hail,severity_rating
2000,704540,ADAK (NAS),AK,51.883,-176.650,+0005.0,12.9,41.957418,69.1,46.22,0.0,0,0,0,0.0
682,997380,ADAK ISLAND,AK,51.870,-176.630,+0007.0,18.7,42.015254,64.9,0.00,0.0,0,0,0,0.0
738,703926,AKHIOK,AK,56.933,-154.183,+0013.0,8.1,37.401887,71.1,9.28,0.0,0,0,0,0.0
547,702686,AKIAK,AK,60.903,-161.231,+0009.1,-20.0,34.427976,75.9,1.45,0.0,0,0,0,0.0
13,999999,ALEKNAGIK 1 NNE,AK,59.284,-158.615,+0024.4,-57.1,52.816638,122.4,809.88,8.0,0,0,0,8.0


## Update js export to work with our JavaScript code ##
For the use with our javascript, we want "let stations_all = " to precede the data

In [16]:
with open("../data/stations_all.js", 'r+') as file:
    # read the contents into a variable
    content = file.read()
    # set cursor to start of file
    file.seek(0, 0)
    # write the desired starter text and rewrite the original content
    file.write("let stations_all = " + content)
    file.close()